## Computing the Sensitivity Indices Using the Output of a Model

In the example used above the Ishigami function is very cheap to
evaluate. However, in most real scenarios the functions of interest are
expensive, and we need to limit ourselves to a few number of
evaluations. Using Monte Carlo methods is infeasible in these scenarios
as a large number of samples are typically required to provide good
estimates of the Sobol indices.

An alternative in these cases is to use Gaussaian process emulator of
the function of interest trained on a few inputs and outputs (Marrel et
al., 2009). If the model is properly trained, its mean prediction which
is cheap to evaluate, can be used to compute the Monte Carlo estimates
of the Sobol indices, the variance from the GP emulator can also be used
to assess our uncertainty about the Sobol indices. Let’s see how we can
do this in Emukit.

We start by generating 100 samples in the input domain. Note that this a
just 1% of the number of samples that we used to compute the Sobol
coefficients using Monte Carlo.

In [56]:
from emukit.core.initial_designs import RandomDesign

In [57]:
design = RandomDesign(space)
x = design.get_samples(500)
y = ishigami.fidelity1(x)[:,np.newaxis]

Now, we fit a standard Gaussian process to the samples, and we wrap it
as an Emukit model.

In [58]:
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.sensitivity.monte_carlo import MonteCarloSensitivity

In [59]:
model_gpy = GPRegression(x,y)
model_emukit = GPyModelWrapper(model_gpy)
model_emukit.optimize()

The final step is to compute the coefficients using the class
`ModelBasedMonteCarloSensitivity` which directly calls the model and
uses its predictive mean to compute the Monte Carlo estimates of the
Sobol indices. We plot the true estimates, those computed using 10000
direct evaluations of the object using Monte Carlo and those computed
using a Gaussian process model trained on 100 evaluations.

## Catapult Simulation

<span class="editsection-bracket"
style="">\[</span><span class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_uq/includes/catapult-simulation.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_uq/includes/catapult-simulation.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

<svg viewBox="0 0 200 200" style="width:15%">

<defs> <clipPath id="clip0">

<style>
circle {
  fill: black;
}
</style>

<circle cx="100" cy="100" r="100"/> </clipPath> </defs>

<title>

Nicolas Durrande

</title>

<image preserveAspectRatio="xMinYMin slice" width="100%" xlink:href="https://mlatcl.github.io/mlphysical/./slides/diagrams//people/nicolas-durrande2.jpg" clip-path="url(#clip0)"/>

</svg>

As a worked example we’re going to introduce a catapult simulation
written by Nicolas Durrande, <https://durrande.shinyapps.io/catapult/>.

<img class="" src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/catapult-simulation.png" style="width:80%">

Figure: <i>A catapult simulation for experimenting with surrogate
models, kindly provided by Nicolas Durrande</i>

The simulator allows you to set various parameters of the catapult
including the axis of rotation, `roation_axis`, the position of the arm
stop, `arm_stop`, and the location of the two bindings of the catapult’s
spring, `spring_binding_1` and `spring_binding_2`.

These parameters are then collated in a vector, $$
\mathbf{ x}_i = \begin{bmatrix}
\texttt{rotation_axis} \\
\texttt{arm_stop} \\
\texttt{spring_binding_1} \\
\texttt{spring_binding_2}
\end{bmatrix}
$$

Having set those parameters, you can run an experiment, by firing the
catapult. This will show you how far it goes.

Because you will need to operate the catapult yourself, we’ll create a
function to query you about the result of an individual firing.

In [60]:
import numpy as np

In [61]:
def catapult_distance(x):
    """Request user input for the catapult."""
    y = np.zeros((x.shape[0], 1))
    for i in range(x.shape[0]):
        rotation_axis=x[i, 0]
        arm_stop=x[i, 1]
        spring_binding_1=x[i, 2]
        spring_binding_2=x[i, 3]
            
        print('Please set the following values:')
        print('x_1 = {rotation_axis:.2f} (rotation axis)'.format(rotation_axis=rotation_axis))
        print('x_2 = {arm_stop:.2f} (arm stop)'.format(arm_stop=arm_stop))
        print('x_3 = {spring_binding_1:.2f} (spring binding 1)'.format(spring_binding_1=spring_binding_1))
        print('x_4 = {spring_binding_2:.2f} (spring binding 2)'.format(spring_binding_2=spring_binding_2))
        y[i, 0] = float(input('What is the distance? '))
    return y

We can also set the parameter space for the model. Each of these
variables is scaled to operate $\in [0, 1]$.

In [62]:
from emukit.core import ContinuousParameter, ParameterSpace

In [63]:
variable_domain = [0,1]
           
space = ParameterSpace(
          [ContinuousParameter('rotation_axis', *variable_domain), 
           ContinuousParameter('arm_stop', *variable_domain),
           ContinuousParameter('spring_binding_1', *variable_domain),
           ContinuousParameter('spring_binding_2', *variable_domain)])

Before we perform sensitivity analysis, we need to build an emulator of
the catapulter, which we do using our experimental design process.

## Experimental Design for the Catapult

<span class="editsection-bracket"
style="">\[</span><span class="editsection"
style=""><a href="https://github.com/lawrennd/snippets/edit/main/_uq/includes/catapult-experimental-design.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/snippets/edit/main/_uq/includes/catapult-experimental-design.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

Now we will build an emulator for the catapult using the experimental
design loop.

We’ll start with a small model-free design, we’ll use a random design
for initializing our model.

In [64]:
from emukit.core.initial_designs import RandomDesign

In [65]:
design = RandomDesign(space)
x = design.get_samples(5)
y = catapult_distance(x)

Please set the following values:
x_1 = 0.34 (rotation axis)
x_2 = 0.35 (arm stop)
x_3 = 0.16 (spring binding 1)
x_4 = 0.14 (spring binding 2)
What is the distance? 20
Please set the following values:
x_1 = 0.26 (rotation axis)
x_2 = 0.03 (arm stop)
x_3 = 0.71 (spring binding 1)
x_4 = 0.13 (spring binding 2)
What is the distance? 30
Please set the following values:
x_1 = 0.01 (rotation axis)
x_2 = 0.18 (arm stop)
x_3 = 0.17 (spring binding 1)
x_4 = 0.03 (spring binding 2)
What is the distance? 34
Please set the following values:
x_1 = 0.57 (rotation axis)
x_2 = 0.15 (arm stop)
x_3 = 0.69 (spring binding 1)
x_4 = 0.85 (spring binding 2)
What is the distance? 40
Please set the following values:
x_1 = 0.28 (rotation axis)
x_2 = 0.04 (arm stop)
x_3 = 0.08 (spring binding 1)
x_4 = 0.78 (spring binding 2)
What is the distance? 50


In [66]:
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.sensitivity.monte_carlo import MonteCarloSensitivity

Set up the GPy model. The variance of the RBF kernel is set to $150^2$
because that’s roughly the square of the range of the catapult. We set
the noise variance to a small value.

In [67]:
model_gpy = GPRegression(x,y)
model_gpy.kern.variance = 150**2
model_gpy.likelihood.variance.fix(1e-5)
display(model_gpy)

GP_regression.,value,constraints,priors
rbf.variance,22500.0,+ve,
rbf.lengthscale,1.0,+ve,
Gaussian_noise.variance,1e-05,+ve fixed,


Wrap the model for EmuKit.

In [68]:
model_emukit = GPyModelWrapper(model_gpy)
model_emukit.optimize()

 C:\Users\sharm\anaconda3\lib\site-packages\paramz\transformations.py:111: RuntimeWarning:overflow encountered in expm1


In [69]:
display(model_gpy)

GP_regression.,value,constraints,priors
rbf.variance,22499.999480775903,+ve,
rbf.lengthscale,5.202834813064715,+ve,
Gaussian_noise.variance,1e-05,+ve fixed,


Now we set up the model loop. We’ll use integrated variance reduction as
the acquisition function for our model-based design loop.

*Warning*: This loop runs much slower on Google `colab` than on a local
machine.

In [70]:
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop

In [71]:
from emukit.experimental_design.acquisitions import IntegratedVarianceReduction, ModelVariance

In [72]:
integrated_variance = IntegratedVarianceReduction(space=space,
                                                  model=model_emukit)
ed = ExperimentalDesignLoop(space=space, 
                            model=model_emukit, 
                            acquisition = integrated_variance)
ed.run_loop(catapult_distance, 10)

Please set the following values:
x_1 = 0.34 (rotation axis)
x_2 = 0.79 (arm stop)
x_3 = 0.58 (spring binding 1)
x_4 = 0.55 (spring binding 2)


KeyboardInterrupt: Interrupted by user

## Sensitivity Analysis of a Catapult Simulation

The final step is to compute the coefficients using the class
`ModelBasedMonteCarloSensitivity` which directly calls the model and
uses its predictive mean to compute the Monte Carlo estimates of the
Sobol indices. We plot the estimates of the Sobol indices computed using
a Gaussian process model trained on the observations we’ve acquired.

In [ ]:
num_mc = 10000
senstivity = MonteCarloSensitivity(model = model_emukit, input_domain = space)
main_effects_gp, total_effects_gp, _ = senstivity.compute_effects(num_monte_carlo_points = num_mc)

In [ ]:
import matplotlib.pyplot as plt
import mlai.plot as plot
import mlai

In [ ]:
import pandas as pd

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)

main_effects_gp_plot = {ivar: main_effects_gp[ivar][0] for ivar in main_effects_gp}

d = {'GP Monte Carlo':main_effects_gp_plot}

pd.DataFrame(d).plot(kind='bar', ax=ax)
plt.ylabel('% of explained output variance')

mlai.write_figure(filename='first-order-sobol-indices-gp-catapult.svg', directory='./uq')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/first-order-sobol-indices-gp-catapult.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>First Order sobol indices as estimated by GP-emulator based
Monte Carlo on the catapult.</i>

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)

total_effects_gp_plot = {ivar: total_effects_gp[ivar][0] for ivar in total_effects_gp}

d = {'GP Monte Carlo':total_effects_gp_plot}

pd.DataFrame(d).plot(kind='bar', ax=ax)
ax.set_ylabel('% of explained output variance')

mlai.write_figure(filename='total-effects-sobol-indices-gp-catapult.svg', directory='./uq')

<img src="https://mlatcl.github.io/mlphysical/./slides/diagrams//uq/total-effects-sobol-indices-gp-catapult.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Total effects as estimated by GP based Monte Carlo on the
catapult.</i>